In [1]:

from bs4 import BeautifulSoup
import yfinance as yf
import pandas as pd
import requests
import warnings
#Future warnings are suppressed
warnings.simplefilter(action='ignore', category=FutureWarning)


#1) DATA COLLECTION PHASE

#1.A)Stock data of Tesla and Gamestop are extracted from Yahoo Finance with yfinance function 

#Data Extraction of Tesla
data=yf.Ticker("TSLA")
tesla_data=data.history(period="max")
tesla_data.reset_index(inplace=True)


#Data Extraction of Gamestop
data=yf.Ticker("GME")
gamestop_data=data.history(period="max")
gamestop_data.reset_index(inplace=True)
gamestop_data.head()

#1.B)Stock data of Tesla and Gamestop are extracted with webscrapting

#Revenue data of Tesla is extracted with using Webscrabting.
url1 = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
html_data  = requests.get(url1).text
soup = BeautifulSoup(html_data, 'html5lib')
tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])

# First we isolate the body of the table which contains all the information
# Then we loop through each row and find all the column values for each row
for row in soup.find("tbody").find_all('tr'):
    col = row.find_all("td")
    date = col[0].text
    revenue= col[1].text
    
    # Finally we append the data of each row to the table
    tesla_revenue = tesla_revenue.append({"Date":date, "Revenue":revenue}, ignore_index=True)    

#Revenue data of Gamestop is extracted with using Webscrabting.
url1 = 'https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue'

html_data  = requests.get(url1).text
soup = BeautifulSoup(html_data, 'html5lib')
gme_revenue = pd.DataFrame(columns=["Date", "Revenue"])

# First we isolate the body of the table which contains all the information
# Then we loop through each row and find all the column values for each row
for row in soup.find("tbody").find_all('tr'):
    col = row.find_all("td")
    date = col[0].text
    revenue= col[1].text
    
    # Finally we append the data of each row to the table
    gme_revenue = gme_revenue.append({"Date":date, "Revenue":revenue}, ignore_index=True)    

In [67]:
#2) DATA PREPERATION PHASE

display(tesla_data.head())
display(gamestop_data.head())


display(gme_revenue.head())
display(tesla_revenue.head())

#Values at Revenue column have dollar signs and commas instead of dots. 
#Data need to be manipulated for numerical calculations such as finding mean value of Revenue.
#For this reason dolar sign ("$") is neglected and commas (",") are replaced with dot (".")

gme_revenue["Revenue"] = gme_revenue['Revenue'].str.replace('|\$',"")
gme_revenue["Revenue"] = gme_revenue['Revenue'].str.replace(',',".")
gme_revenue.dropna(inplace=True)
gme_revenue = gme_revenue[gme_revenue['Revenue'] != ""] 

tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace('|\$',"")
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',',".")
tesla_revenue.dropna(inplace=True)
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""] 

display(gme_revenue.head())
display(tesla_revenue.head())

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620128,1.693350,1.603296,1.691666,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716074,1.670626,1.683250,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658002,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615920,1.662210,1.603296,1.662210,6892800,0.0,0.0


,Date,Revenue
0,2022,6.011
1,2021,5.090
2,2020,6.466
3,2019,8.285
4,2018,8.547


,Date,Revenue
0,2021,53.823
1,2020,31.536
2,2019,24.578
3,2018,21.461
4,2017,11.759


C:\Users\xboxm\AppData\Local\Temp\ipykernel_10856\1231406762.py:14: FutureWarning:

The default value of regex will change from True to False in a future version.

C:\Users\xboxm\AppData\Local\Temp\ipykernel_10856\1231406762.py:19: FutureWarning:

The default value of regex will change from True to False in a future version.



,Date,Revenue
0,2022,6.011
1,2021,5.090
2,2020,6.466
3,2019,8.285
4,2018,8.547


,Date,Revenue
0,2021,53.823
1,2020,31.536
2,2019,24.578
3,2018,21.461
4,2017,11.759


In [64]:
#3) DATA VISUALIZATION PHASE

#Visualization function is created
import plotly.graph_objects as go
from plotly.subplots import make_subplots
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

#Desired outputs are created.
make_graph(tesla_data, tesla_revenue, 'Tesla')
make_graph(gamestop_data, gme_revenue, 'Gamestop')